# Representing text

If we want to solve Natural Language Processing (NLP) tasks with neural networks, we need some way to represent text as tensors. Computers already represent characters as numbers that map to letters on your screen using encodings such as ASCII or UTF-8.

如果我们想用神经网络解决自然语言处理 (NLP) 任务，我们需要某种方法将文本表示为张量。 计算机已经将字符表示为使用 ASCII 或 UTF-8 等编码映射到屏幕上字母的数字。

![Image showing diagram mapping a character to an ASCII and binary representation](./images/ascii-character-map.png)

We understand what each letter **represents**, and how all characters come together to form the words of a sentence. However, computers don't have such an understanding, and neural networks have to learn the meaning of the sentence during training.

We can use different approaches when representing text:
* **Character-level representation**, where we represent text by treating each character as a number. Given that we have $C$ different characters in our text corpus, the word *Hello* could be represented by a tensor with shape $C \times 5$. Each letter would correspond to a tensor in one-hot encoding.
* **Word-level representation**, in which we create a **vocabulary** of all words in our text, and then represent words using one-hot encoding. This approach is better than character-level representation because each letter by itself does not have much meaning. By using higher-level semantic concepts - words - we simplify the task for the neural network. However, given a large dictionary size, we need to deal with high-dimensional sparse tensors.

Let's start by installing some required Python packages we'll use in this module.

我们了解每个字母**代表**，以及所有字符如何组合成一个句子的单词。 然而，计算机没有这样的理解，神经网络必须在训练过程中学习句子的含义。

我们可以在表示文本时使用不同的方法：
* **字符级表示**，我们通过将每个字符视为一个数字来表示文本。 鉴于我们的文本语料库中有 $C$ 个不同的字符，单词 *Hello* 可以用形状为 $C\times 5$ 的张量表示。 每个字母都对应于 one-hot 编码中的一个张量。
* **词级表示**，我们在其中创建文本中所有词的**词汇表**，然后使用单热编码表示词。 这种方法优于字符级表示，因为每个字母本身没有太多意义。 通过使用更高层次的语义概念——单词——我们简化了神经网络的任务。 然而，给定一个大字典大小，我们需要处理高维稀疏张量。

让我们首先安装我们将在本模块中使用的一些必需的 Python 包。

In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0

# Text classification task

In this module, we will start with a simple text classification task based on the **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)** dataset: we'll classify news headlines into one of 4 categories: World, Sports, Business and Sci/Tech. To load the dataset, we will use the **[TensorFlow Datasets](https://www.tensorflow.org/datasets)** API.

> In the sandbox environment, we need to pre-fetch the dataset from a known location before creating it with TensorFlow datasets. If you're running in your local environment, you can skip the next cell, and the TensorFlow datasets library will download the data automatically.

在本模块中，我们将从基于 **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)** 数据集的简单文本分类任务开始：我们将对新闻进行分类 标题分为 4 个类别之一：世界、体育、商业和科技。 要加载数据集，我们将使用 **[TensorFlow 数据集](https://www.tensorflow.org/datasets)** API。

> 在沙盒环境中，我们需要在使用 TensorFlow 数据集创建数据集之前从已知位置预取数据集。 如果您在本地环境中运行，则可以跳过下一个单元格，TensorFlow 数据集库将自动下载数据。

In [2]:
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [3]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

We can now access the training and test portions of the dataset by using `dataset['train']` and `dataset['test']` respectively:

我们现在可以分别使用 `dataset['train']` 和 `dataset['test']` 访问数据集的训练和测试部分：

In [4]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

120000
7600


Let's print out the first 10 new headlines from our dataset: 

让我们打印出数据集中的前 10 个新标题：

In [5]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Text vectorization

Now we need to convert text into **numbers** that can be represented as tensors. If we want word-level representation, we need to do two things:

* Use a **tokenizer** to split text into **tokens**.
* Build a **vocabulary** of those tokens.

现在我们需要将文本转换为可以表示为张量的**数字**。 如果我们想要词级表示，我们需要做两件事：

* 使用 **tokenizer** 将文本拆分为 **tokens**。
* 构建这些标记的**词汇表**。

### Limiting vocabulary size

In the AG News dataset example, the vocabulary size is rather big, more than 100k words. Generally speaking, we don't need words that are rarely present in the text &mdash; only a few sentences will have them, and the model will not learn from them. Thus, it makes sense to limit the vocabulary size to a smaller number by passing an argument to the vectorizer constructor:

Both of those steps can be handled using the **TextVectorization** layer. Let's instantiate the vectorizer object, and then call the `adapt` method to go through all text and build a vocabulary:

在 AG News 数据集示例中，词汇量相当大，超过 100k 个单词。 一般来说，我们不需要文本中很少出现的词—— 只有几句话会有它们，模型不会从中学习。 因此，通过将参数传递给向量化器构造函数来将词汇表大小限制为较小的数字是有意义的：

这两个步骤都可以使用 **TextVectorization** 层来处理。 让我们实例化 vectorizer 对象，然后调用 adapt 方法遍历所有文本并构建词汇表：



In [16]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Note** that we are using only subset of the whole dataset to build a vocabulary. We do it to speed up the execution time and not keep you waiting. However, we are taking the risk that some of the words from the whole dateset would not be included into the vocabulary, and will be ignored during training. Thus, using the whole vocabulary size and running through all dataset during `adapt` should increase the final accuracy, but not significantly.

Now we can access the actual vocabulary:

> **注意**我们只使用整个数据集的子集来构建词汇表。 我们这样做是为了加快执行时间，而不是让您久等。 但是，我们冒着这样的风险，即整个数据集中的某些单词不会包含在词汇表中，并且在训练过程中会被忽略。 因此，在“适应”期间使用整个词汇表大小并遍历所有数据集应该会提高最终的准确性，但不会显着。

现在我们可以访问实际的词汇表：

In [17]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'of', 'in', 'and', 'on', 'for']
Length of vocabulary: 50000


Using the tokenizer, we can easily encode any text into a set of numbers:

In [18]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 372, 2297,    3,  312,   12, 1293, 2314])>

## Bag-of-words text representation

Because words represent meaning, sometimes we can figure out the meaning of a piece of text by just looking at the individual words, regardless of their order in the sentence. For example, when classifying news, words like *weather* and *snow* are likely to indicate *weather forecast*, while words like *stocks* and *dollar* would count towards *financial news*.

**Bag-of-words** (BoW) vector representation is the most simple to understand traditional vector representation. Each word is linked to a vector index, and a vector element contains the number of occurrences of each word in a given document.

因为单词代表意义，所以有时我们只看单个单词就可以弄清楚一段文字的意思，而不管它们在句子中的顺序如何。 例如，在对新闻进行分类时，*weather* 和 *snow* 等词可能表示 *weather forecast*，而 *stocks* 和 *dollar* 等词可能表示 *financial news*。

**Bag-of-words** (BoW) 向量表示是最简单易懂的传统向量表示。 每个词都链接到一个向量索引，向量元素包含给定文档中每个词的出现次数。

![Image showing how a bag of words vector representation is represented in memory.](./images/bag-of-words-example.png) 

> **Note**: You can also think of BoW as a sum of all one-hot-encoded vectors for individual words in the text.

Below is an example of how to generate a bag-of-words representation using the Scikit Learn python library:

> **注意**：您也可以将 BoW 视为文本中单个单词的所有 one-hot-encoded 向量的总和。

下面是如何使用 Scikit Learn python 库生成词袋表示的示例：

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()



array([[1, 1, 0, 2, 0, 0, 0, 0, 0]])

We can also use the Keras vectorizer that we defined above, converting each word number into a one-hot encoding and adding all those vectors up:

我们还可以使用上面定义的 Keras 向量化器，将每个单词编号转换为 one-hot 编码并将所有这些向量相加：

In [20]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

> **Note**: You may be surprised that the result differs from the previous example. The reason is that in the Keras example the length of the vector corresponds to the vocabulary size, which was built from the whole AG News dataset, while in the Scikit Learn example we built the vocabulary from the sample text on the fly. 


> **注意**：您可能会惊讶于结果与前面的示例不同。 原因是在 Keras 示例中，向量的长度对应于从整个 AG News 数据集构建的词汇量大小，而在 Scikit Learn 示例中，我们从动态样本文本构建词汇表。

## Training the BoW classifier

Now that we have learned how to build the bag-of-words representation of our text, let's train a classifier that uses it. First, we need to convert our dataset to a bag-of-words representation. This can be achieved by using `map` function in the following way:

现在我们已经学习了如何构建文本的词袋表示，让我们训练一个使用它的分类器。 首先，我们需要将我们的数据集转换为词袋表示。 这可以通过以下方式使用 `map` 函数来实现：

In [21]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Now let's define a simple classifier neural network that contains one linear layer. The input size is `vocab_size`, and the output size corresponds to the number of classes (4). Because we're solving a classification task, the final activation function is **softmax**:

现在让我们定义一个包含一个线性层的简单分类器神经网络。 输入大小为“vocab_size”，输出大小对应于类数 (4)。 因为我们正在解决分类任务，所以最终的激活函数是 **softmax**：

In [22]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 88s 94ms/step - loss: 0.5466 - acc: 0.8759 - val_loss: 0.3682 - val_acc: 0.8950


Since we have 4 classes, an accuracy of above 80% is a good result.

由于我们有 4 个类，因此准确率超过 80% 是一个不错的结果。

## Training a classifier as one network

Because the vectorizer is also a Keras layer, we can define a network that includes it, and train it end-to-end. This way we don't need to vectorize the dataset using `map`, we can just pass the original dataset to the input of the network.

> **Note**: We would still have to apply maps to our dataset to convert fields from dictionaries (such as `title`, `description` and `label`) to tuples. However, when loading data from disk, we can build a dataset with the required structure in the first place.

因为向量器也是一个 Keras 层，我们可以定义一个包含它的网络，并端到端地训练它。 这样我们就不需要使用 `map` 对数据集进行矢量化，我们只需将原始数据集传递给网络的输入即可。

> **注意**：我们仍然需要将映射应用于我们的数据集，以将字典中的字段（例如`title`、`description`和`label`）转换为元组。 但是，当从磁盘加载数据时，我们可以首先构建具有所需结构的数据集。

In [23]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_6 (TextVe (None, None)              0         
_________________________________________________________________
tf_op_layer_OneHot (TensorFl [(None, None, 50000)]     0         
_________________________________________________________________
tf_op_layer_Sum (TensorFlowO [(None, 50000)]           0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 200004    
Total params: 200,004
Trainable params: 200,004
Non-trainable params: 0
_________________________________________________________________
938/938 [==============================] - 79s 84ms/step - loss: 0.5221 - acc: 0.8804 - val_loss: 0.3447 - val_acc

## Bigrams, trigrams and n-grams

One limitation of the bag-of-words approach is that some words are part of multi-word expressions, for example, the word 'hot dog' has a completely different meaning from the words 'hot' and 'dog' in other contexts. If we represent the words 'hot' and 'dog' always using the same vectors, it can confuse our model.

To address this, **n-gram representations** are often used in methods of document classification, where the frequency of each word, bi-word or tri-word is a useful feature for training classifiers. In bigram representations, for example, we will add all word pairs to the vocabulary, in addition to original words.

Below is an example of how to generate a bigram bag of word representation using Scikit Learn:

词袋方法的一个局限性是某些词是多词表达的一部分，例如，“hot dog”一词与其他语境中的“hot”和“dog”一词具有完全不同的含义。 如果我们总是使用相同的向量表示“hot”和“dog”这两个词，它可能会混淆我们的模型。

为了解决这个问题，**n-gram 表示**经常用于文档分类方法，其中每个词、双词或三词的频率是训练分类器的有用特征。 例如，在二元表示中，除了原始单词之外，我们还将所有单词对添加到词汇表中。

下面是一个示例，说明如何使用 Scikit Learn 生成词表示的二元组包：

In [24]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

The main drawback of the n-gram approach is that the vocabulary size starts to grow extremely fast. In practice, we need to combine the n-gram representation with a dimensionality reduction technique, such as *embeddings*, which we will discuss in the next unit.

To use an n-gram representation in our **AG News** dataset, we need to pass the `ngrams` parameter to our `TextVectorization` constructor. The length of a bigram vocaculary is **significantly larger**, in our case it is more than 1.3 million tokens! Thus it makes sense to limit bigram tokens as well by some reasonable number.

We could use the same code as above to train the classifier, however, it would be very memory-inefficient. In the next unit, we will train the bigram classifier using embeddings. In the meantime, you can experiment with bigram classifier training in this notebook and see if you can get higher accuracy.

n-gram 方法的主要缺点是词汇量开始增长得非常快。 在实践中，我们需要将 n-gram 表示与降维技术相结合，例如 *embeddings*，我们将在下一单元中讨论。

要在我们的 **AG News** 数据集中使用 n-gram 表示，我们需要将 `ngrams` 参数传递给我们的 `TextVectorization` 构造函数。 bigram vocaculary 的长度**明显更大**，在我们的例子中它超过了 130 万个标记！ 因此，将二元组标记也限制在一些合理的数量上是有意义的。

我们可以使用与上面相同的代码来训练分类器，但是，它的内存效率非常低。 在下一个单元中，我们将使用嵌入来训练二元分类器。 同时，您可以在本笔记本中尝试二元分类器训练，看看是否可以获得更高的准确率。

## Automatically calculating BoW Vectors

In the example above we calculated BoW vectors by hand by summing the one-hot encodings of individual words. However, the latest version of TensorFlow allows us to calculate BoW vectors automatically by passing the `output_mode='count` parameter to the vectorizer constructor. This makes defining and training our model significanly easier:

在上面的示例中，我们通过对单个单词的单热编码求和来手动计算 BoW 向量。 然而，最新版本的 TensorFlow 允许我们通过将 `output_mode='count` 参数传递给向量化器构造函数来自动计算 BoW 向量。 这使得定义和训练我们的模型变得更加容易：

In [25]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 10s 11ms/step - loss: 0.5207 - acc: 0.8826 - val_loss: 0.3430 - val_acc: 0.9051


## Term frequency - inverse document frequency (TF-IDF)

In BoW representation, word occurrences are weighted using the same technique regardless of the word itself. However, it's clear that frequent words such as *a* and *in* are much less important for classification than specialized terms. In most NLP tasks some words are more relevant than others.

**TF-IDF** stands for **term frequency - inverse document frequency**. It's a variation of bag-of-words, where instead of a binary 0/1 value indicating the appearance of a word in a document, a floating-point value is used, which is related to the frequency of the word occurrence in the corpus.

More formally, the weight $w_{ij}$ of a word $i$ in the document $j$ is defined as:

在 BoW 表示中，无论单词本身如何，都使用相同的技术对单词出现进行加权。 但是，很明显，*a* 和 *in* 等常用词在分类方面的重要性远低于专业术语。 在大多数 NLP 任务中，有些词比其他词更相关。

**TF-IDF** 代表 **term frequency - inverse document frequency**。 它是词袋的变体，其中使用浮点值代替二进制 0/1 值指示文档中单词的出现，这与语料库中单词出现的频率有关 .

更正式地说，文档 $j$ 中单词 $i$ 的权重 $w_{ij}$ 定义为：

$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
where
* $tf_{ij}$ is the number of occurrences of $i$ in $j$, i.e. the BoW value we have seen before
* $N$ is the number of documents in the collection
* $df_i$ is the number of documents containing the word $i$ in the whole collection

The TF-IDF value $w_{ij}$ increases proportionally to the number of times a word appears in a document and is offset by the number of documents in the corpus that contains the word, which helps to adjust for the fact that some words appear more frequently than others. For example, if the word appears in *every* document in the collection, $df_i=N$, and $w_{ij}=0$, and those terms would be completely disregarded.

You can easily create TF-IDF vectorization of text using Scikit Learn:

其中
* $tf_{ij}$是$i$在$j$中出现的次数，即我们之前看到的BoW值
* $N$ 是集合中文档的数量
* $df_i$ 是整个集合中包含单词 $i$ 的文档数

TF-IDF 值 $w_{ij}$ 与一个词在文档中出现的次数成正比增加，并被语料库中包含该词的文档数抵消，这有助于调整一些词出现的事实 比其他人出现得更频繁。 例如，如果单词出现在集合中的*每个*文档中，$df_i=N$ 和 $w_{ij}=0$，那么这些术语将被完全忽略。

您可以使用 Scikit Learn 轻松创建文本的 TF-IDF 向量化：

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

In Keras, the `TextVectorization` layer can automatically compute TF-IDF frequencies by passing the `output_mode='tf-idf'` parameter. Let's repeat the code we used above to see if using TF-IDF increases accuracy: 

在 Keras 中，`TextVectorization` 层可以通过传递 `output_mode='tf-idf'` 参数自动计算 TF-IDF 频率。 让我们重复上面使用的代码，看看使用 TF-IDF 是否可以提高准确性：

In [21]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 94s 101ms/step - loss: 0.3203 - acc: 0.9039 - val_loss: 0.2542 - val_acc: 0.9186


## Conclusion 

Even though TF-IDF representations provide frequency weights to different words, they are unable to represent meaning or order. As the famous linguist J. R. Firth said in 1935, "The complete meaning of a word is always contextual, and no study of meaning apart from context can be taken seriously." We will learn how to capture contextual information from text using language modeling in a later unit.

尽管 TF-IDF 表示为不同的词提供了频率权重，但它们无法表示含义或顺序。 正如著名语言学家 J. R. Firth 在 1935 年所说的那样，“一个词的完整含义总是与上下文相关的，离开上下文对意义的研究是不能认真对待的。” 我们将在后面的单元中学习如何使用语言建模从文本中捕获上下文信息。